# End-to-End LLM based solution
Using sentence semantic symilarity and summaring models

## Tests and Tries

In [1]:
import numpy as np

In [2]:
COLAB = True

In [3]:
if COLAB:
    from google.colab import drive
    from os.path import exists

    amazon_train_file = 'train.ft.txt.bz2'

    if exists(amazon_train_file):
        train_file = amazon_train_file
    else:
        drive.mount('/content/drive/')

        # For Hadas' drive
        my_dir = 'drive/MyDrive/Y-data/Intuit-K-anonimity/'

        # For Lior's drive
        #my_dir = 'drive/MyDrive/Y-data/Y-DATA_PROJECT/'

        train_file = my_dir + '/train.ft.txt.bz2'
else:
    train_file = '../data/' + 'train.ft.txt.bz2'


Mounted at /content/drive/


In [4]:
# Credit https://www.kaggle.com/code/anshulrai/cudnnlstm-implementation-93-7-accuracy

import bz2

# Readling the file to list of comments
train_file = bz2.BZ2File(train_file)
train_file_lines = train_file.readlines()

# Converting from raw binary strings to strings that can be parsed
train_file_lines = [x.decode('utf-8') for x in train_file_lines]

# Extracting the labels and sentences
#train_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in train_file_lines]
train_sentences = [x.split(' ', 1)[1][:-1].lower() for x in train_file_lines] # And converting to lower case

del(train_file_lines)  # Free RAM


In [5]:
len(train_sentences)

3600000

### Working with short sentences - temporarly

In [6]:
short_train_sentences = [x for x in train_sentences if len(x.split(' ')) < 20]
len(short_train_sentences)

28005

In [7]:
train_sentences, all_sentences = short_train_sentences, train_sentences

### Sentences similarity

In [8]:
!pip install sentence_transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.1 MB/s eta 0:00:00


In [9]:
from sentence_transformers import SentenceTransformer

Embedding

In [10]:
%%time
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v1')
embeddings = model.encode(train_sentences[:100])


CPU times: user 8.79 s, sys: 362 ms, total: 9.15 s
Wall time: 9.43 s


Finding the most similar sentences to the first sentence

In [11]:
one_sent = embeddings[0]
sim = []
idx, max_idx = 1, 1
for idx, e in enumerate(embeddings[:]):
    cos_sim = np.dot(e, one_sent)/(np.linalg.norm(e)*np.linalg.norm(one_sent))
    sim.append((cos_sim, idx))

sim.sort(reverse=True)
sim[0:4]

[(1.0000001, 0), (0.6311436, 32), (0.55656093, 45), (0.46512488, 67)]

In [12]:
train_sentences[0]

'textbook: book shipped quickly and was in excellent condition as stated. easy transaction would buy again'

In [13]:
train_sentences[32]

'recent purchase: the book i ordered was exactly as described and delivery was even faster than promised.'

Finding K nearest neighbors using Annoy

In [14]:
!pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.4/647.4 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.2-cp310-cp310-linux_x86_64.whl size=582735 sha256=7bbc17a18c3b7d30399eef4f6ffaf862913f17f4374412a606b85a032ec92a83
  Stored in directory: /root/.cache/pip/wheels/7a/d9/59/473fa56df8e39430eeda369500b4e7127f5b243ba24c3c4297
Successfully built annoy


In [15]:
k = 3

In [16]:
from annoy import AnnoyIndex

# Build an Annoy index with 10 trees. angular = cosine similarity
annoy_index = AnnoyIndex(embeddings.shape[1], metric='angular')
for i, x in enumerate(embeddings):
    annoy_index.add_item(i, x)
annoy_index.build(10)

# Find the k nearest neighbors to the first sentence
nearest_neighbors = annoy_index.get_nns_by_vector(embeddings[0], k)

print('Nearest neighbors:', nearest_neighbors)


Nearest neighbors: [0, 32, 45]


In [17]:
embeddings

array([[-0.0036093 ,  0.05210925, -0.12970327, ..., -0.02006356,
        -0.04063172,  0.02199112],
       [-0.0052765 ,  0.06117417, -0.03607288, ..., -0.00327409,
         0.02298493,  0.00951664],
       [ 0.05052014,  0.05758742, -0.08671991, ..., -0.03691006,
         0.05248274,  0.05065772],
       ...,
       [-0.0529007 , -0.02020332,  0.01042636, ...,  0.01338089,
         0.01423867,  0.01405939],
       [ 0.04678291, -0.04269627,  0.12175517, ..., -0.01851123,
         0.04590768, -0.03081301],
       [-0.04308654, -0.05753473, -0.03191447, ..., -0.02193826,
         0.0352149 ,  0.03438053]], dtype=float32)

In [18]:
print(train_sentences[0])
print(train_sentences[32])
print(train_sentences[45])

textbook: book shipped quickly and was in excellent condition as stated. easy transaction would buy again
recent purchase: the book i ordered was exactly as described and delivery was even faster than promised.
recommend this seller: i recieved this book on time and in excellent condition. i'd definitely recommend this seller.


### Summaring multiple sentences into one

In [19]:
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# model_name = "snrspeaks/t5-one-line-summary"
# sum_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

In [20]:
#sum_text([train_sentences[0], train_sentences[32], train_sentences[45]], sum_model, tokenizer)

## Putting it all together - function definition

In [21]:
%%time
emb_model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v1')

CPU times: user 377 ms, sys: 42.1 ms, total: 419 ms
Wall time: 375 ms


In [79]:
def print_example(indexes, origina_docs, new_docs):
    print('Before:')
    for i in indexes:
        print(origina_docs[i])
    print('\nAfter:')
    for i in indexes:
        print(new_docs[i])
    

In [23]:
def find_k_nearesr_neighbors(emb, emb_list, k):
    """ Find the K nearest neighbors using Annoy package """

    # Build an Annoy index with 10 trees. angular = cosine similarity
    annoy_index = AnnoyIndex(emb_list.shape[1], metric='angular')
    for i, x in enumerate(emb_list):
        annoy_index.add_item(i, x)
    annoy_index.build(10)

    # Find the k nearest neighbors to the first sentence
    nearest_neighbors = annoy_index.get_nns_by_vector(emb, k)

    return nearest_neighbors

In [68]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-large")

def sum_text(doc_list):
    # define the input sentences
    #input_text = '. '.join(doc_list)
    input_text = ''
    i = 1
    for doc in doc_list:
       input_text = f'{input_text}{i}: {doc}. ' 
       i += 1

    # preprocess the input sentences
    input_ids = tokenizer.encode(f'summarize:' + input_text, return_tensors="pt")

    # generate the summary sentence
    output_ids = model.generate(input_ids=input_ids, max_length=32, num_beams=4, early_stopping=True)
    output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return output

In [76]:
def run_anonymization_on_txt(docs, k):
    """ Finding K nearest neighbors and summarize them """
    annon_docs = docs.copy()
    used_indexes = set([])
    
    # Embedding
    docs_emb = emb_model.encode(docs)
    temp_docs_emb = docs_emb.copy()

    for i, d in enumerate(docs):
        #print('i:', i, '\t', used_indexes)
        # To prevent redandent
        if i not in used_indexes:
            #used_indexes.add(i)  # Adding to the used items
            similar_doc_ind = find_k_nearesr_neighbors(temp_docs_emb[i], temp_docs_emb, k)
            
            curr_docs = []
            for sd in similar_doc_ind:
                if sd in used_indexes:
                    print('Error: the index', sd, 'was already used')
                # Adding the document to the similar doc list
                curr_docs.append(docs[sd])
                # Adding the index to the used items
                used_indexes.add(sd)  
                # Prevent repeating comparison by changing the vector
                temp_docs_emb[sd] = 1000 * np.random.randint(10, size=len(temp_docs_emb[sd]))
                #temp_docs_emb[sd] = [1000] * len(temp_docs_emb[sd])
            sum_doc = sum_text(curr_docs)
            print('similar_doc_ind', similar_doc_ind, '\tSummary:', sum_doc)
            #print('sum_doc:', sum_doc)
            for sd in similar_doc_ind:
                annon_docs[sd] = sum_doc
        if  len(used_indexes) > (len(docs) - k):
            print('Breaking! \tlen(used_indexes)', len(used_indexes), '\tlen(docs)', len(docs), '\tlen(docs)-k', (len(docs) - k))
            break
    return annon_docs



## Running on 100 examples

In [77]:
%%time
annon_sents_100 = run_anonymization_on_txt(train_sentences[:100], k=3)

similar_doc_ind [0, 32, 45] 	Summary: the book shipped quickly and was in excellent condition as stated. i'd definitely recommend this seller.
similar_doc_ind [1, 72, 73] 	Summary: janes all the worlds aircraft 1996-7: great deal, very quick delivery. when i received the bike, the packing board was broken.
similar_doc_ind [2, 63, 41] 	Summary: edge of danger: 1 star - only because that's the minimum. the write-up was not good. this is definitely a children
similar_doc_ind [3, 49, 56] 	Summary: maeve's forte is character development, dialogue, and interaction. there wasn't enough romance though. great performances by all of the
similar_doc_ind [4, 35, 47] 	Summary: frida's more attractive soundtrack sister: utterly amazing and unique sound. america the beautiful: this book works great for teaching the
similar_doc_ind [5, 6, 80] 	Summary: wishmaster is a fantastic album, that should grace everyone's music collection. innovating and always refreshing. truly a masterpiece.
similar_doc_ind 

Output examples

In [80]:
print_example([0, 32, 45], train_sentences, annon_sents_100)

Before:
textbook: book shipped quickly and was in excellent condition as stated. easy transaction would buy again
recent purchase: the book i ordered was exactly as described and delivery was even faster than promised.
recommend this seller: i recieved this book on time and in excellent condition. i'd definitely recommend this seller.

After:
the book shipped quickly and was in excellent condition as stated. i'd definitely recommend this seller.
the book shipped quickly and was in excellent condition as stated. i'd definitely recommend this seller.
the book shipped quickly and was in excellent condition as stated. i'd definitely recommend this seller.


In [82]:
print_example([1, 72, 73], train_sentences, annon_sents_100)

Before:
janes all the worlds aircraft 1996-7: great to deal with. very quick delivery. very highly recommended. thank you.
good bike, bad packing: when i received the bike, the packing board was broken.the bike quality is good.
great purchase: installed in seconds. great performance. excellent to eliminate phone line dependency.

After:
janes all the worlds aircraft 1996-7: great deal, very quick delivery. when i received the bike, the packing board was broken.
janes all the worlds aircraft 1996-7: great deal, very quick delivery. when i received the bike, the packing board was broken.
janes all the worlds aircraft 1996-7: great deal, very quick delivery. when i received the bike, the packing board was broken.


In [83]:
print_example([19, 78, 87], train_sentences, annon_sents_100)

Before:
beach boys and the satan: a totally off and enjoyable movie. all brian wilson fans should see this movie.
great: one of adams best films along side the wedding singer and mr. deeds. funny movie. worth buying
solid follow up: very good movie. for children 7 and up...any younger they may not grasp the concept.

After:
beach boys and the satan: a totally off and enjoyable movie. one of adams best films along side the wedding singer
beach boys and the satan: a totally off and enjoyable movie. one of adams best films along side the wedding singer
beach boys and the satan: a totally off and enjoyable movie. one of adams best films along side the wedding singer


Test anonymity

In [56]:
#get_pesonal_docs(annon_sents)

## Running on 1000 examples

In [86]:
%%time
annon_sents_1000 = run_anonymization_on_txt(train_sentences[:1001], k=3)

similar_doc_ind [0, 546, 933] 	Summary: used library book, but still in good condition. book came quickly and was cheap. overall good deal!
similar_doc_ind [1, 200, 713] 	Summary: janes all the worlds aircraft 1996-7: great to deal with. very quick delivery. very highly recommended.
similar_doc_ind [2, 526, 934] 	Summary: this claptrap would have never been published had the author not been black and female. the plot lines are distinctly implausible 
similar_doc_ind [3, 367, 989] 	Summary: the movie has a very weak storyline, loosely linked chapters, total disappointment from a good cast. it was definitely not a classic
similar_doc_ind [4, 243, 285] 	Summary: i love enya's work and this cd is exceptional. my son is thoroughly enjoying this audio version of the music found
similar_doc_ind [5, 518, 126] 	Summary: the things this man can do is amazing. this cd is incredible. exciting the whole way through.
similar_doc_ind [6, 312, 773] 	Summary: wishmaster is a fantastic album, that shoul

In [99]:
# For running on 1000 documents
print_example([54, 750, 324], train_sentences, annon_sents_1000)

Before:
boring and hard to watch/listen: waste of money. i purchased all 7 and gave them away. hard to watch.
boring: i found myself not caring about the main character and i didn't find the musings very entertaining.
waste of time: what a disappointment. nothing like my favorite, the green mile. dont waste your time. boring.

After:
i purchased all 7 and gave them away. hard to watch/listen: waste of money. i found myself not caring about the main character and
i purchased all 7 and gave them away. hard to watch/listen: waste of money. i found myself not caring about the main character and
i purchased all 7 and gave them away. hard to watch/listen: waste of money. i found myself not caring about the main character and


In [92]:
print_example([1, 619, 200], train_sentences, annon_sents_1000)

Before:
janes all the worlds aircraft 1996-7: great to deal with. very quick delivery. very highly recommended. thank you.
multi-engine flying: great book for multi-pilots, also includes all necessary info for oral and practical tests.
great service: great customer service and shipping was quick product in great condition. love the movie. thanks.

After:
janes all the worlds aircraft 1996-7: great to deal with. very quick delivery. very highly recommended.
i'm interested in early flight and this book gave an eyewitness account which i enjoyed. the night before christmas: excellent book.
janes all the worlds aircraft 1996-7: great to deal with. very quick delivery. very highly recommended.


In [88]:
print_example([2, 526, 934], train_sentences, annon_sents_1000)

Before:
edge of danger: 1 star - only because that's the minimum.this book proves the famous can publish anything.
one star too many: this claptrap would have never been published had the author not been black and female.
implausible ...: this author writes well enough ... but the plot lines are distinctly implausible ... a disappointment.

After:
this claptrap would have never been published had the author not been black and female. the plot lines are distinctly implausible 
this claptrap would have never been published had the author not been black and female. the plot lines are distinctly implausible 
this claptrap would have never been published had the author not been black and female. the plot lines are distinctly implausible 


In [89]:
print_example([3, 367, 989], train_sentences, annon_sents_1000)

Before:
a disappointment: maeve's forte is character development, dialogue, and interaction, and silver wedding was far below her usual standard.
a disappointment: the movie has a very weak storyline, loosely linked chapters, total disappointment from a good cast.
disappointment: this movie was a disappointment compared to babe. it was definitely not a classic sort of movie!

After:
the movie has a very weak storyline, loosely linked chapters, total disappointment from a good cast. it was definitely not a classic
the movie has a very weak storyline, loosely linked chapters, total disappointment from a good cast. it was definitely not a classic
the movie has a very weak storyline, loosely linked chapters, total disappointment from a good cast. it was definitely not a classic


In [90]:
print_example([5, 518, 126], train_sentences, annon_sents_1000)

Before:
awesome.: what can i say about this album that hasn't already been said? nothing. awesome album on every level.
excellent: the things this man can do is amazing. this cd is incredible. exciting the whole way through.
another hit: nothing like anything moby has done before, it is a great album still, i highly recommend it.

After:
the things this man can do is amazing. this cd is incredible. exciting the whole way through.
the things this man can do is amazing. this cd is incredible. exciting the whole way through.
the things this man can do is amazing. this cd is incredible. exciting the whole way through.


In [94]:
print_example([72, 712, 159], train_sentences, annon_sents_1000)

Before:
good bike, bad packing: when i received the bike, the packing board was broken.the bike quality is good.
terrible bag: i purchased this bag and returned it. it is very disappointing in appearance, materials and workmanship.
poor quality: the seat covers started coming loose at the seams immediately upon putting them on the seats.

After:
when i received the bike, the packing board was broken. the seat covers started coming loose at the seams immediately upon putting them on 
when i received the bike, the packing board was broken. the seat covers started coming loose at the seams immediately upon putting them on 
when i received the bike, the packing board was broken. the seat covers started coming loose at the seams immediately upon putting them on 


In [93]:
print_example([86, 349, 117], train_sentences, annon_sents_1000)

Before:
horrible book: boring and bland. he even spent two pages listing sponges. don't waste your time by reading this.
average and plain.: this book is boring not funny and defenitely no worth to read. don't waste your time
boring: this book was totally boring. even though it was 1 of the greatest books i've read.hahahahaha

After:
horrible book: boring and bland. he even spent two pages listing sponges. don't waste your time by reading this.
horrible book: boring and bland. he even spent two pages listing sponges. don't waste your time by reading this.
horrible book: boring and bland. he even spent two pages listing sponges. don't waste your time by reading this.


### Another example for summary

In [65]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-large")

In [67]:
# define the input sentences
input_text = "1: textbook: book shipped quickly and was in excellent condition as stated. easy transaction would buy again. 2: recent purchase: the book i ordered was exactly as described and delivery was even faster than promised. 3: recommend this seller: i received this book on time and in excellent condition. i'd definitely recommend this seller."
#input_text = "disappointed: this is what happens when artists get too ambitious, ohh man i'm gonna miss the old nelly. curious, so i bought it: another trying hard wannabe dance diva. she should stick to singing ballads. tangerine dream is great.: i love tangerine dream, but this album is not my favorite. still worth owning though."
input_text = "1: janes all the worlds aircraft 1996-7: great to deal with. very quick delivery. very highly recommended. thank you. 2: good bike, bad packing: when i received the bike, the packing board was broken.the bike quality is good. 3: great purchase: installed in seconds. great performance. excellent to eliminate phone line dependency."

# preprocess the input sentences
input_ids = tokenizer.encode("summarize:" + input_text, return_tensors="pt")

# generate the summary sentence
output_ids = model.generate(input_ids=input_ids, max_length=32, num_beams=4, early_stopping=True)
output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(output)


janes all the worlds aircraft 1996-7: great deal, very quick delivery. when i received the bike, the packing board was broken.


## Test anonymity

### Define functions

In [ ]:
# Credit: https://www.kaggle.com/code/pierremegret/gensim-word2vec-tutorial 

import spacy

nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed

def cleaning(doc):
    # Defining the document
    doc = nlp(doc) 

    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]
    
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    #if len(txt) > 2:
    #    return ' '.join(txt)
    clean_doc = ' '.join(txt)
    return clean_doc

In [ ]:
import re

def clean_corpus(corpus):
    """ Cleans the corpus """
    brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in corpus)
    corpus_lemmas = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000)]
    return corpus_lemmas

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer is defined only once
vectorizer = CountVectorizer(ngram_range=(1,1), # to use bigrams ngram_range=(2,2)
                           stop_words='english')

def get_anonym_degree(docs, min_k = None):
    """ If K not given, returns the minimal current k and the corresponding documents.
        If k is given, return the documents with k or less neighbohrs  """
    
    # Lemmatizing the documents
    ldocs = clean_corpus(docs)

    # Vectorizing
    count_data = vectorizer.fit_transform(ldocs)
    count_data = count_data.toarray()
    # Converting any number larger than 1 into 1
    count_data[count_data > 1] = 1
    # Counting unique values
    uniq_arr, uniq_cnt = np.unique(count_data, axis=0, return_counts=True)
    if not min_k:
        min_k = min(uniq_cnt)
        # All the unique vectors
        un_anon = uniq_arr[uniq_cnt <= min_k]
    else:
        # All the unique vectors
        un_anon = uniq_arr[uniq_cnt < min_k]
        min_k = min(uniq_cnt) # For the return value

    # Getting the unique vectore indeces
    indeces_list = []
    for row in un_anon:
        # Get the similar rows
        similar_vals = np.where((count_data == (row)).all(axis=1))
        indeces_list.append(similar_vals[0].tolist())

    return min_k, indeces_list

### Running

In [ ]:
get_anonym_degree(annon_sents)

In [ ]:
get_anonym_degree(annon_sents, min_k=3)

## GPT-Chat output

**Input**:   
* janes all the worlds aircraft 1996-7: great to deal with. very quick delivery. very highly recommended. thank you.
* good bike, bad packing: when i received the bike, the packing board was broken.the bike quality is good.
* great purchase: installed in seconds. great performance. excellent to eliminate phone line dependency.

**Output**:   
"The reviews indicate positive experiences with quick delivery and great performance of the products, but with some issues in packaging."

____

**Input**:   
* textbook: book shipped quickly and was in excellent condition as stated. easy transaction would buy again
* recent purchase: the book i ordered was exactly as described and delivery was even faster than promised.
* recommend this seller: i recieved this book on time and in excellent condition. i'd definitely recommend this seller.

**Output**:   
"The reviews are positive and recommend the seller for their fast shipping and excellent condition of the books."
____

**Input**:   
* beach boys and the satan: a totally off and enjoyable movie. all brian wilson fans should see this movie.
* great: one of adams best films along side the wedding singer and mr. deeds. funny movie. worth buying
* solid follow up: very good movie. for children 7 and up...any younger they may not grasp the concept.

**Output**:   
"The reviews are positive and recommend the movies for their enjoyable and entertaining qualities."		